# Uniform search and A*-search using `aima-python`

Algorithm implementations taken from [here](https://github.com/aimacode/aima-python/blob/master/search-4e.ipynb)

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
from time import time
from copy import deepcopy

import numpy as np
np.random.seed(2016)

In [36]:
from search import Problem, astar_search, uniform_cost_search
from santas_bags_problem_2 import fill_all_bags, SantasBagsProblem2
import sys
sys.path.append('../common')
from utils import weight3 as weight_fn, weight_by_index
from utils import score2
from utils import MAX_WEIGHT, AVAILABLE_GIFTS_LIST, GIFT_TYPES, N_TYPES, N_BAGS

In [37]:
GIFT_TYPES

['ball', 'bike', 'blocks', 'book', 'coal', 'doll', 'gloves', 'horse', 'train']

In [38]:
fixed_weights = [
    1.99876912083, # ball
    20.0021364556, # bike
    11.6630321858, # blocks
    2.00086596571, # book
    23.7866257713, # coal
    4.9993625282,  # doll
    1.40310067709, # gloves
    4.99527064522, # horse
    10.0234458084  # train
]

In [39]:
total_weight = 0
for i, g in enumerate(GIFT_TYPES):
    total_weight += fixed_weights[i] * AVAILABLE_GIFTS_LIST[i]
total_weight

50511.0646006388

In [40]:
def bag_weight(state, count=100):
    w = []
    for c in range(count):
        m = 0
        for i, v in enumerate(state):
            if v > 0:
                m += np.sum([weight_by_index(i) for j in range(v)])
        w.append(m)
    return np.mean(w)

In [389]:
alpha = 0.75
goal_score = MAX_WEIGHT*alpha
print("Goal score : ", goal_score)
initial_state = empty_state

p = SantasBagsProblem2(initial=initial_state,
                      available_gifts=AVAILABLE_GIFTS_LIST,
                      max_weight=MAX_WEIGHT,    
                      bag_weight_fn=bag_weight,
                      goal_score=goal_score, 
                      score_fn=score2,
                      verbose_level=1)

# uniform_cost_search(p)

Goal score :  37.5


## Heuristic functions + A*-search

In [53]:
empty_state = tuple([0]*N_TYPES)

In [515]:
# Penalize states that heavy than goal score 
# def h1(state, limit):   
#     w = bag_weight(state)
#     if w < limit:
#         return (limit - w)**2 / limit**2
#     else:
#         return 1.0

    
def h1(state, limit):   
    w = score2(state)
    return (MAX_WEIGHT - w)/MAX_WEIGHT
    
    
# Count number of rejected bags :
def h2(state):
    _, rejected = score2(state, return_rejected=True)
    return 3.0 * rejected

In [516]:
from search import FrontierPQ, Node

In [517]:
def create_problem(state, available_gifts, **kwargs):
    return SantasBagsProblem2(initial=state,
                      available_gifts=available_gifts,
                      max_weight=MAX_WEIGHT,    
                      bag_weight_fn=bag_weight,
                      goal_score=goal_score, 
                      score_fn=score2,
                      verbose_level=0)


In [518]:
goal_score = 0.75 * n_bags_per_state*MAX_WEIGHT
limit = goal_score + 0.5*(MAX_WEIGHT-goal_score)

p = create_problem(empty_state, AVAILABLE_GIFTS_LIST, goal_score=goal_score)

costfn=lambda node: node.path_cost + h1(node.state, MAX_WEIGHT) + h2(node.state)

frontier = FrontierPQ(Node(p.initial), costfn)
explored = set()

In [587]:
node = frontier.pop()
print("- State:", node.state, 
        "score", score2(node.state),
        "h1: ", h1(node.state, limit), "h2: ", h2(node.state))
if p.is_goal(node.state):
    print(">>> Goal found : ", node.state)
explored.add(node.state)
for action in p.actions(node.state):
    child = node.child(p, action)
    print("-- State:", child.state, 
          "score", score2(child.state),
          "h1: ", h1(child.state, limit), "h2: ", h2(child.state))
    if child.state not in explored and child.state not in frontier:
        frontier.add(child)
    elif child.state in frontier:
        incumbent = frontier.states[child.state]
        if child.path_cost < frontier.costfn(incumbent):
            frontier.replace(child)


- State: (3, 1, 0, 0, 0, 0, 0, 2, 0) score 31.3196630014 h1:  0.369187823539 h2:  0.21
-- State: (4, 1, 0, 0, 0, 0, 0, 2, 0) score 29.8798146194 h1:  0.373325326907 h2:  0.3
-- State: (3, 1, 1, 0, 0, 0, 0, 2, 0) score 24.6830416326 h1:  0.480449009418 h2:  1.17
-- State: (3, 1, 0, 1, 0, 0, 0, 2, 0) score 31.1980634363 h1:  0.457124348058 h2:  0.36
-- State: (3, 1, 0, 0, 0, 1, 0, 2, 0) score 31.7176379095 h1:  0.384889149456 h2:  0.72
-- State: (3, 1, 0, 0, 0, 0, 1, 2, 0) score 30.1726344347 h1:  0.398309169653 h2:  0.24
-- State: (3, 1, 0, 0, 0, 0, 0, 3, 0) score 30.2465702197 h1:  0.425111285563 h2:  0.6
-- State: (3, 1, 0, 0, 0, 0, 0, 2, 1) score 24.5564880968 h1:  0.493288406658 h2:  1.14


In [110]:
limit = goal_score + 0.5*(MAX_WEIGHT-goal_score)

def final_heuristic_fn(state):
    return h1(state, limit) + h2(state)
    

def astar_search_algo(problem, **kwargs):
    return astar_search(problem, final_heuristic_fn, **kwargs)
    

def update_problem(p, state, available_gifts,
                    counter, total_state,
                    found_goal_states,
                    **kwargs):
    p.initial = state
    p.available_gifts = available_gifts
    
        
def termination_condition(p, counter, total_state):
    if p.goal_score < 18:
        return True
    return False
    

In [111]:
astar_search_algo(p)

KeyboardInterrupt: 

In [ ]:
fill_all_bags(create_fix_weight_problem, astar_search_algo, 
                total_state, found_goal_states, available_gifts, counter,
                score,
                update_problem_fn=update_problem,
                termination_condition_fn=termination_condition
             )

In [ ]:
def to_submission(state, available_gifts, gift_types):
    n_gifts = [available_gifts[t] for t in gift_types]
    output = []
    for bag in state:
        o = []
        for index, count in enumerate(bag):   
            gift_type = gift_types[index]
            for i in range(count):
                v = n_gifts[index] - 1
                assert v >= 0, "Gift index is negative"
                o.append(gift_type + '_%i' % v)
                n_gifts[index] -= 1
        output.append(o)  
    return output
        
submission = to_submission(total_state, AVAILABLE_GIFTS, GIFT_TYPES)
# print(submission)

In [ ]:
from datetime import datetime
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

In [ ]:
def write_submission(state, filename):
    with open(filename, 'w') as w:
        w.write("Gifts\n")
        for bag in state:
            w.write(' '.join(bag) + '\n')
    
write_submission(submission, submission_file)